In [91]:
import pandas as pd
import sys,os,copy,pdb,importlib
sys.path.append('../classes')
sys.path.append('../analysis')
import numpy as np
import matplotlib.pyplot as plt

import tournament_new as tn
import create_database as cd
importlib.reload(tn)

import beck.beck_game
from importlib import reload
reload(beck.beck_game)
from beck.beck_game import BeckGame as Game
game = Game(4,9,4)

from arena import Arena
importlib.reload(tn)
game = Game(4,9,4)
all_p = pd.read_pickle(cd.DATABASE_LOC)

# res = tn.merge_res_to_base()
tournament_res = pd.read_pickle('/scratch/zz737/fiar/tournaments/ai_all_player_round_robin_base.pkl')

import argparse
import os
import shutil
import time
import random
import numpy as np
import math
import sys

import keras
import keras.backend as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import metrics

sys.path.append('..')
from neural_net import NeuralNet
from utils import *

reload(beck.beck_nnet)
from beck.beck_nnet import OthelloNNet, NNetWrapper

In [30]:
args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 10,
    'batch_size': 64,
    'cuda': False,
    'num_channels': 512,
})

Onnet =OthelloNNet(game,args)


In [42]:
def masked_mean(x,mask, **kwargs):
    mask = K.cast(mask, K.floatx())
    return K.sum(x * mask, **kwargs) / K.sum(mask, **kwargs)
    
def masked_std(x, mask, **kwargs):
    mask = K.cast(mask, K.floatx())
    masked_x = x * mask
    x_mean = masked_mean(x,mask,**kwargs)
    var = masked_mean( (x - x_mean)**2, mask, **kwargs)
    std = tf.math.sqrt(var)
    return std

def masked_standardize(x, mask, **kwargs):
    return (x - masked_mean(x, mask, keepdims=True,**kwargs)) / masked_std(x, mask, keepdims=True,**kwargs)

class Correlation(keras.layers.Layer):
    def __init__(self):
        super(Correlation,self).__init__()
    
    def call(self, policy, val_reshaped, valids):
        '''

        policy: b x action_size
        val_reshaped: b x action_size
        
        valids: b x 36; for masking out the occupied indices in correlation computation; valids = game.getValidMovesBatch(input, 1)
        '''

        p_stddz = masked_standardize(policy, valids, axis=1)
        val_stddz = masked_standardize(val_reshaped, valids, axis=1)
        corr = masked_mean(p_stddz * val_stddz, valids, axis=1, keepdims=False)
        
        mask = ~tf.math.is_nan(corr)
        return corr[mask] # if one var is constant, ignore this data

In [102]:
import tensorflow as tf
import tensorflow.math as tfm

class OthelloNNet_sub():
    def __init__(self, game, args):
        # game params
        self.board_x, self.board_y = game.getBoardSize()
        self.action_size = game.getActionSize()
        self.args = args

        # Neural Net
        self.input_boards = Input(shape=(self.board_x, self.board_y))    # s: batch_size x board_x x board_y
        
        x_image = Reshape((self.board_x, self.board_y, 1))(self.input_boards)                # batch_size  x board_x x board_y x 1
        h_conv1 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(args.num_channels, 3, padding='same', use_bias=False)(x_image)))         # batch_size  x board_x x board_y x num_channels
        h_conv2 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(args.num_channels, 3, padding='same', use_bias=False)(h_conv1)))         # batch_size  x board_x x board_y x num_channels
        h_conv3 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(args.num_channels, 3, padding='valid', use_bias=False)(h_conv2)))        # batch_size  x (board_x-2) x (board_y-2) x num_channels
        h_conv4 = Activation('relu')(BatchNormalization(axis=3)(Conv2D(args.num_channels, 2, padding='valid', use_bias=False)(h_conv3)))        # batch_size  x (board_x-4) x (board_y-4) x num_channels
        h_conv4_flat = Flatten()(h_conv4)       
        s_fc1 = Dropout(args.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(1024, use_bias=False)(h_conv4_flat))))  # batch_size x 1024
        s_fc2 = Dropout(args.dropout)(Activation('relu')(BatchNormalization(axis=1)(Dense(512, use_bias=False)(s_fc1))))          # batch_size x 1024
        self.pi = Dense(self.action_size, activation='softmax', name='pi')(s_fc2)   # batch_size x self.action_size
        self.v = Dense(1, activation='tanh', name='v')(s_fc2)                    # batch_size x 1
        
        self.model = Model(inputs=self.input_boards, outputs=[self.pi, self.v])


class OthelloNNet_with_PVcorr(Model):
    def __init__(self,game,args=None,sub_model=None,**kwargs):
        super(OthelloNNet_with_PVcorr, self).__init__()
        self.game=game
        self.action_size = game.getActionSize()
        self.args = args
        if sub_model is None:
            if args is None:
                print('args cannot be None')
                return
            else:
                self.sub_model = OthelloNNet_sub(game,self.args).model
        else:
            self.sub_model = sub_model
            
        

        self.corr = Correlation()
        
    def call(self,inputs):
        '''
        input: b x game.m x game.n
        input_expanded: (b x action_size) x game.m x game.n
        valids: b x 36; for masking out the occupied indices in correlation computation; valids = game.getValidMovesBatch(input, 1)
        '''
        inputs_expanded,valids = self.get_next_step_boards(self.game, inputs)
        B = inputs.shape[0]
        pi,v_train  = self.sub_model(inputs) # b x action_size
        _,v_expanded = self.sub_model(inputs_expanded) #  (b x action_size) x 1
        v_expanded_reshaped = K.reshape(v_expanded,(B, self.action_size)) # b x action_size
        corr_mask = self.corr(pi, v_expanded_reshaped, valids)
#         return pi, v_train, corr_mask

        return corr_mask
    
    @staticmethod
    def get_next_step_boards(game,board_batch):
        '''
        [SZ]
        board_batcch: b x game.m x game.n
        input_expanded: (b x action_size) x game.m x game.n
        valids: b x action_size
        '''
        board_batch = tf.cast(board_batch,tf.int32)
        valids = tf.cast(K.reshape((board_batch==0),(board_batch.shape[0],-1)),tf.float32)
        action_size = game.getActionSize()
        input_expanded = K.repeat_elements(board_batch,action_size, axis=0) # repeat for : 1 2 -> 1 1 2 2; in tf2.7, K.repeat is good
        inds_to_be_placed = K.tile(K.arange(action_size),board_batch.shape[0]) # tile for: 1 2 -> 1 2 1 2
        xs, ys = inds_to_be_placed // game.n, inds_to_be_placed % game.n
        expanded_batch_inds = K.arange(input_expanded.shape[0])
        inds_tensor = tf.stack([expanded_batch_inds,xs,ys],axis=1)      
        input_expanded_updated = tf.tensor_scatter_nd_update(input_expanded, inds_tensor,tf.ones(input_expanded.shape[0],dtype=tf.int32))

        return tf.stop_gradient(input_expanded_updated), tf.stop_gradient(valids)


In [103]:
class Scaled_MSE_with_mask(keras.losses.Loss):
    def __init__(self, scale):
        super().__init__()
        self.scale = scale
    def call(self, y_true, y_pred):
        '''
        y_true: b, not actually used. Fixed at 1.
        y_pred: corr, masked; cannot put tuple here! 
        '''
        res = self.scale * keras.losses.mean_squared_error(1, y_pred)
        
        return res
#         return tf.where(tf.math.is_nan(res),tf.zeros_like(res),res) # if nan, ie mask all False, then return 0?

In [105]:
onpv = OthelloNNet_with_PVcorr(game,sub_model=None,args=args)

optim = keras.optimizers.Adam(learning_rate=1e-3)
sc = 1
# onpv.compile(optimizer=optim, loss=['categorical_crossentropy','mean_squared_error',Scaled_MSE_with_mask(sc)])
onpv.compile(optimizer=optim, loss=[Scaled_MSE_with_mask(sc)])
# onpv.compile(optimizer=optim, loss=['mean_squared_error'])

In [6]:
from pickle import Pickler, Unpickler
weight_dir = '/scratch/zz737/fiar/tournaments/tournament_6/checkpoints_mcts100_cpuct2_id_0'
fn = 'checkpoint_8.pth.tar'
load_folder_file = (weight_dir,fn)

modelFile = os.path.join(load_folder_file[0], load_folder_file[1])
examplesFile = modelFile + ".examples"
with open(examplesFile, "rb") as f:
    trainExamplesHistory = Unpickler(f).load()


from random import shuffle
trainExamples = []
for e in trainExamplesHistory:
    trainExamples.extend(e)
shuffle(trainExamples)

input_boards, target_pis, target_vs = list(zip(*trainExamples[:10]))
input_boards = np.asarray(input_boards)
target_pis = np.asarray(target_pis)
target_vs = np.asarray(target_vs)

In [67]:
import beck.beck_nnet_pvcorr as bbnp
reload(bbnp)
onpv = bbnp.OthelloNNet_with_PVcorr(game,sub_model=None,args=bbnp.args)

In [68]:
optim = keras.optimizers.Adam(learning_rate=1e-3)
sc = 1
onpv.compile(optimizer=optim, loss=['categorical_crossentropy','mean_squared_error',bbnp.Scaled_MSE_with_mask(sc)])
# onpv.compile(optimizer=optim, loss=[None,None,bbnp.Scaled_MSE_with_mask(sc)])
# onpv.compile(optimizer=optim, loss=['categorical_crossentropy','mean_squared_error',None])
# onpv.compile(optimizer=optim, loss=[None,'mean_squared_error',None], metrics=[None,metrics.MeanSquaredError(),None])


# onpv.sub_model.compile(optimizer=optim, loss=['categorical_crossentropy','mean_squared_error'])


In [69]:
keras.losses.get('mean_squared_error')(target_vs,onpv(input_boards,training=True)[1][:,0])

<tf.Tensor: shape=(), dtype=float32, numpy=1.8577244>

In [70]:

onpv.fit(x=input_boards, y=[target_pis, target_vs, target_vs],batch_size=10,epochs=10)
# onpv.sub_model.fit(x=input_boards, y=[target_pis, target_vs],batch_size=10,epochs=10)

# onpv.fit(x=input_boards, y=[target_vs],batch_size=1,epochs=100)

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 7.4185 - output_1_loss: 4.2121 - output_2_loss: 2.2384 - output_3_loss: 0.9680
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 5.3142 - output_1_loss: 3.7876 - output_2_loss: 0.5801 - output_3_loss: 0.9464
Epoch 3/10
1/1 [==============================] - 0s 1ms/step - loss: 3.8856 - output_1_loss: 2.8865 - output_2_loss: 0.2153 - output_3_loss: 0.7838
Epoch 4/10
1/1 [==============================] - 0s 1ms/step - loss: 4.2435 - output_1_loss: 2.7692 - output_2_loss: 0.7391 - output_3_loss: 0.7351
Epoch 5/10
1/1 [==============================] - 0s 1ms/step - loss: 3.4362 - output_1_loss: 2.5695 - output_2_loss: 0.1271 - output_3_loss: 0.7396
Epoch 6/10
1/1 [==============================] - 0s 3ms/step - loss: 3.6000 - output_1_loss: 2.4748 - output_2_loss: 0.3804 - output_3_loss: 0.7447
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 3.4644 - output_1_loss: 2.3903 - out

In [75]:
target_pis[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.25252525, 0.        , 0.51515152,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.13131313, 0.        , 0.1010101 , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [90]:
onpv(input_boards,training=False)[0].numpy()[0]

array([0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.02, 0.02, 0.01, 0.02, 0.01,
       0.02, 0.12, 0.02, 0.18, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.06,
       0.02, 0.06, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.03,
       0.02, 0.01, 0.01], dtype=float32)

In [85]:
onpv.sub_model.predict(input_boards)

[array([[0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.02, 0.02, 0.01, 0.02, 0.02,
         0.02, 0.11, 0.02, 0.17, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.06,
         0.02, 0.06, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.03,
         0.02, 0.02, 0.01],
        [0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.02, 0.02, 0.01, 0.02, 0.02,
         0.02, 0.11, 0.02, 0.16, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.06,
         0.02, 0.06, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.03,
         0.02, 0.02, 0.01],
        [0.02, 0.02, 0.02, 0.01, 0.02, 0.02, 0.02, 0.02, 0.01, 0.02, 0.02,
         0.02, 0.11, 0.02, 0.16, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.06,
         0.02, 0.06, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.03,
         0.02, 0.02, 0.01],
        [0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.02, 0.02, 0.01, 0.02, 0.02,
         0.02, 0.11, 0.02, 0.16, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.06,
         0.02, 0.06, 0.03, 0.02, 0.02, 0.02, 0.02, 0.01, 0.01, 0.02, 0.03,
         0.02, 0

In [80]:
np.set_printoptions(precision=2,suppress=True)
onpv(input_boards,training=True)[2]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.58],
       [0.36],
       [0.07],
       [0.08],
       [0.02],
       [0.06],
       [0.1 ],
       [0.61],
       [0.18],
       [0.12]], dtype=float32)>

In [62]:
# keras.losses.get('categorical_crossentropy')(target_pis,onpv(input_boards)[0])
keras.losses.get('mean_squared_error')(target_vs,onpv(input_boards,training=False)[1][:,0])
# keras.losses.get('mean_squared_error')(target_vs,onpv.sub_model(input_boards)[1][:,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.900448>

In [39]:
target_vs

array([-1,  1,  1, -1, -1, -1,  1, -1,  1,  1])

In [38]:
Onnet.model(input_boards,training=True)[1]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-0.999391  ],
       [ 0.98747796],
       [ 0.91292125],
       [-0.9596786 ],
       [-0.9940864 ],
       [-0.99351805],
       [ 0.9940994 ],
       [ 0.9951146 ],
       [ 0.99927   ],
       [ 0.9999735 ]], dtype=float32)>

In [33]:
Onnet.model.fit(x=input_boards,y=[target_pis, target_vs],batch_size=10,epochs=5)

Epoch 1/5
1/1 [==============================] - 0s 1ms/step - loss: 5.2372 - pi_loss: 4.2217 - v_loss: 1.0155
Epoch 2/5
1/1 [==============================] - 0s 1ms/step - loss: 4.1661 - pi_loss: 3.6045 - v_loss: 0.5616
Epoch 3/5
1/1 [==============================] - 0s 1ms/step - loss: 3.4850 - pi_loss: 2.7235 - v_loss: 0.7615
Epoch 4/5
1/1 [==============================] - 0s 1ms/step - loss: 2.8734 - pi_loss: 2.4708 - v_loss: 0.4027
Epoch 5/5
1/1 [==============================] - 0s 3ms/step - loss: 2.7630 - pi_loss: 2.3678 - v_loss: 0.3953


In [35]:
target_vs

array([-1,  1,  1, -1, -1, -1,  1, -1,  1,  1])

In [34]:
Onnet.model(input_boards)[1]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-0.18301925],
       [ 0.221788  ],
       [ 0.08570548],
       [-0.132572  ],
       [-0.14168672],
       [-0.15501614],
       [ 0.116411  ],
       [ 0.221788  ],
       [ 0.13507147],
       [ 0.17073628]], dtype=float32)>

In [40]:
keras.losses.mse(target_vs,Onnet.model(input_boards,training=True)[1][:,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.39779636>

In [182]:
onpv(input_boards)[1]

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[-1.       ],
       [-1.       ],
       [-1.       ],
       [-1.       ],
       [ 1.       ],
       [ 0.4602403],
       [-1.       ],
       [ 1.       ],
       [-1.       ],
       [-1.       ]], dtype=float32)>

In [157]:
# onpv(input_boards)
onpv.loss[-1](1,onpv(input_boards)[-1])

<tf.Tensor: shape=(), dtype=float32, numpy=nan>

In [114]:
onpv.trainable_weights

[<tf.Variable 'conv2d_68/kernel:0' shape=(3, 3, 1, 512) dtype=float32, numpy=
 array([[[[ 0.00026425,  0.01914321, -0.01654135, ..., -0.01557587,
            0.02332167, -0.01153632]],
 
         [[ 0.03460835,  0.04038749, -0.04285367, ..., -0.00353124,
            0.03151251,  0.00452753]],
 
         [[ 0.04965595, -0.02099263, -0.00038375, ...,  0.03043477,
            0.0117238 ,  0.04868086]]],
 
 
        [[[ 0.02820176,  0.00201074,  0.05164923, ...,  0.02080365,
           -0.02787932, -0.02427578]],
 
         [[-0.01132751,  0.03624711, -0.0106726 , ...,  0.05040328,
           -0.04615475,  0.044338  ]],
 
         [[-0.0438347 ,  0.0442006 , -0.02474859, ...,  0.04564536,
            0.00899046, -0.00652765]]],
 
 
        [[[-0.02615054, -0.0256636 ,  0.03686564, ..., -0.0034737 ,
           -0.0009491 , -0.01969924]],
 
         [[ 0.0263382 , -0.00238665,  0.00071868, ...,  0.04618445,
           -0.02661338, -0.00170247]],
 
         [[-0.02624576, -0.00142349, -0.0036

In [34]:
res = keras.losses.mean_squared_error(1, [])

In [37]:
import tensorflow as tf
tf.where(tf.math.is_nan(res),tf.zeros_like(res, dtype=tf.float32),res)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [38]:
onpv.loss[-1](1,onpv(input_boards)[-1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [188]:
keras.losses.get(onpv.loss[0])(target_pis,onpv(input_boards)[0])

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([3.7154484, 2.7252007, 3.1613944, 2.9390788, 3.2853801, 3.9644985,
       3.4503381, 3.614789 , 1.6224487, 3.2527738], dtype=float32)>

In [200]:
((onpv(input_boards)[1].numpy() - target_vs)**2).mean(axis=-1)

array([1.19978066, 1.19998291, 1.19985868, 1.19908755, 1.19967284,
       1.19987363, 1.19979811, 1.19868735, 1.19990709, 1.19994593])

In [205]:
((onpv(input_boards)[1].numpy()[:,0] - target_vs)**2)

array([3.34185621e-08, 2.02934558e-10, 1.38718406e-08, 3.99695717e+00,
       3.99890931e+00, 1.10925988e-08, 3.99932697e+00, 1.19875722e-06,
       5.99485261e-09, 2.03050377e-09])

In [ ]:
target_vs

In [201]:
keras.losses.mean_squared_error(target_vs,onpv(input_boards)[1][:,0])

<tf.Tensor: shape=(), dtype=float32, numpy=1.1995194>

In [190]:
# onpv.loss[-1](target_vs,onpv(input_boards)[-1])
keras.losses.mean_squared_error(target_vs,onpv(input_boards)[1])

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([1.1997806, 1.1999829, 1.1998587, 1.1990875, 1.1996727, 1.1998736,
       1.199798 , 1.1986874, 1.1999071, 1.1999459], dtype=float32)>

In [191]:
onpv.sub_model(input_boards)

[<tf.Tensor: shape=(10, 36), dtype=float32, numpy=
 array([[3.25185582e-02, 1.58983786e-02, 1.06054563e-02, 2.59660929e-02,
         1.73005015e-02, 4.51426627e-03, 3.04146409e-02, 8.15025810e-03,
         1.93655230e-02, 1.63000934e-02, 1.41236801e-02, 8.72853491e-03,
         1.82052940e-01, 1.69135239e-02, 4.63101408e-03, 1.22929782e-01,
         7.90343955e-02, 6.70405990e-03, 2.31358986e-02, 1.08832424e-03,
         6.87595457e-02, 5.09849079e-02, 1.56866424e-02, 2.63217166e-02,
         2.05095261e-02, 1.03331851e-02, 1.40894363e-02, 1.85627975e-02,
         3.02294195e-02, 1.45235739e-04, 1.86227169e-03, 2.27631931e-03,
         1.31086484e-02, 1.06294698e-03, 8.04697424e-02, 5.22163464e-03],
        [2.45335661e-02, 9.81720351e-03, 4.74363379e-03, 1.33887269e-02,
         1.50805963e-02, 3.71309067e-03, 5.52710742e-02, 1.71360541e-02,
         6.85581705e-03, 4.71709995e-03, 2.64888369e-02, 8.97738710e-03,
         1.03226416e-01, 1.06243649e-02, 1.09034276e-03, 7.68808648e-02,

In [192]:
target_vs

array([ 1,  1,  1, -1, -1,  1, -1,  1,  1,  1])

In [241]:
onpv(input_boards)

(<tf.Tensor: shape=(2, 36), dtype=float32, numpy=
 array([[0.02967529, 0.02105615, 0.03391905, 0.01946679, 0.02990824,
         0.01563064, 0.03320701, 0.02909921, 0.02624647, 0.01901171,
         0.01750068, 0.03244977, 0.02467673, 0.01914717, 0.04704242,
         0.0239544 , 0.02170204, 0.02213574, 0.02933961, 0.02786277,
         0.02814098, 0.02568423, 0.02165742, 0.03093002, 0.04971066,
         0.02462929, 0.02655158, 0.03343961, 0.03087012, 0.0600877 ,
         0.01597618, 0.0293151 , 0.02781406, 0.03377112, 0.01578455,
         0.02260546],
        [0.03192541, 0.02492702, 0.03513719, 0.02354246, 0.02805576,
         0.01428352, 0.02836437, 0.02841829, 0.03127316, 0.01702992,
         0.0183426 , 0.03358467, 0.02021199, 0.02119658, 0.04603348,
         0.02812072, 0.0208363 , 0.01979854, 0.02782594, 0.03157841,
         0.02717211, 0.02931771, 0.02028   , 0.02970653, 0.04675816,
         0.02456388, 0.02610142, 0.02940745, 0.03175835, 0.05332172,
         0.01557704, 0.03288532

In [242]:
input_boards

array([[[ 0,  0,  0,  0, -1,  0,  0,  0,  0],
        [ 0,  0,  0,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0]],

       [[ 0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, -1,  0,  1,  0,  1,  0],
        [ 0, -1,  0,  1,  1, -1,  0,  0,  0],
        [ 0,  0,  0,  0,  0, -1,  0,  0,  0]]])

In [329]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
train_dataset = tf.data.Dataset.from_tensor_slices({'input':input_boards,'pi':target_pis})
# train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)


In [331]:
train_dataset

<TensorSliceDataset shapes: {input: (4, 9), pi: (36,)}, types: {input: tf.int64, pi: tf.float64}>

In [330]:
len(list(train_dataset))

35900

In [ ]:
def train_step(self, data):
    # Unpack the data. Its structure depends on your model and
    # on what you pass to `fit()`.
    x, y = data

    with tf.GradientTape() as tape:
        y_pred = self(x, training=True)  # Forward pass
        # Compute the loss value
        # (the loss function is configured in `compile()`)
        loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

    # Compute gradients
    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)
    # Update weights
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    # Update metrics (includes the metric that tracks the loss)
    self.compiled_metrics.update_state(y, y_pred)
    # Return a dict mapping metric names to current value
    return {m.name: m.result() for m in self.metrics}